In [1]:
model_id = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
quantized_model_id = model_id.replace("Qwen/", "saxman/") + "-bnb-fp4"
model_path = "../output/models/" + quantized_model_id

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype="bfloat16",
    llm_int8_skip_modules=[
        "lm_head",
    ],
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config_4bit,
    dtype="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
model.save_pretrained(model_path)
_ = tokenizer.save_pretrained(model_path)

In [ ]:
%timeit

from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

/home/saxman/devel/genscai/.venv/lib/python3.12/site-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub(quantized_model_id)